In [2]:
import h5py
lan = h5py.File('/Users/yupeili/Desktop/ISO/data/mosi.hdf5')

In [42]:
#la = h5py.File('/Users/yupeili/Desktop/ISO/data/mosi_unalign.hdf5')

In [98]:
len(lan['FACET_4.2'].keys())

2183

In [3]:
import numpy as np
video = dict()
for i in lan['FACET_4.2'].keys():
    #replace the last ']' to sort
    video[i] = np.average(lan['FACET_4.2'][i]['features'], axis = 0) #use average value of each words as the final representation

In [4]:
#sort out the video dict
import functools
#aa={'aa[2]':3, 'aa[10]':5, 'aa[0]':10}
#print(aa)
def compare(x, y):
    a = 0
    b = 0
    #j = 0
    #print(x[:3])
    if x[0][:3] > y[0][:3]:
        #print('1')
        return 1
    elif x[0][:3] < y[0][:3]:
        #print('2')
        return -1
    else:
        #print('3')    
        try:
            a = int(x[0][-3:-1])
        except:
            a = int(x[0][-2:-1])
        try:
            b = int(y[0][-3:-1])
        except:
            b = int(y[0][-2:-1])
        #print(a,b)
        if a>b:
            #print('True')
            return 1
        elif a<b:
            return -1
        else:
            return 0
video = sorted(video.items(), key=functools.cmp_to_key(compare))

In [83]:
trainset = dict()
with open('/Users/yupeili/Desktop/ISO/data/train_filename.txt', 'r') as f:
    for lines in f:
        #print(lines.strip()[1:-6])
        trainset[lines.strip()[1:-6]] = 0

testset = dict()
with open('/Users/yupeili/Desktop/ISO/data/test_filename.txt', 'r') as f:
    for lines in f:
        #print(lines.strip()[:-4])
        testset[lines.strip()[:-4]] = 0

In [90]:
for i in video:
    kk = i[0][:i[0].index('[')]
    if kk in trainset.keys():
        trainset[kk] = trainset[kk] + 1
    else:
        testset[kk] = testset[kk] + 1

In [97]:
testset.values()

31

In [98]:
trainset.values()

61

In [87]:
#check the dimension of text and visual
import os
tt = '/Users/yupeili/Desktop/ISO/data/Raw/Transcript/Segmented/'
ft = os.listdir(tt)
for i in ft:
    count = 0
    #print(i)
    with open(tt + i, 'r') as f:
        for lines in f:
            if i[:-15] in trainset.keys():
                trainset[i[:-15]] -= 1
            elif i[:-15] in testset.keys():
                testset[i[:-15]] -= 1

In [270]:
trainset['c5xsKMxpXnc']

-15

In [271]:
testset['2WGyTLYerpo']

-1

add blank data in '2WGyTLYerpo' with its average of 2 data nearby it and throw away 'c5xsKMxpXnc'

In [9]:
for i in range(len(video)):
    if video[i][0] == '2WGyTLYerpo[42]':
        video.insert(i + 1, ('2WGyTLYerpo[43]',((video[i][1]+ video[i+1][1])/2)) )

In [119]:
video_train = np.empty([61, 63, 35], dtype = float)
video_test = np.empty([31, 63, 35], dtype = float)

index_map = dict()
count = 0
for i in trainset.keys():
    index_map[i] = count
    count += 1
for i in testset.keys():
    index_map[i] = count
    count += 1

In [121]:
video_test

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [122]:
judge = True
count = 0
max_count = 0
index = 0
train_test = True

num = 0

for i in video:
    tt = i[0][:i[0].index('[')]
    if judge:
        if tt in trainset.keys():
            #print(1)
            max_count = trainset[tt]
            index = index_map[tt]
            train_test = True
        elif tt in testset.keys():
            #print(2)
            max_count = testset[tt]
            index = index_map[tt] - 61
            train_test = False
        else:
            print('error!!')
        judge = False
        #print(tt)
        #print(index)
        #print(max_count)
    if count < max_count:
        
        if train_test:
            #print(3)
            video_train[index][count] = i[1]
        else:
            #print(4)
            #print(count)
            video_test[index][count] = i[1]
        count += 1
        if count == max_count:
            judge = True
            count = 0
            num += 1
    #if num > 1:
     #   break
        

In [138]:
import pickle
file = open('video_train.pickle', 'wb')
pickle.dump(video_train, file)
file.close()

In [144]:
import pickle
file = open('video_test.pickle', 'wb')
pickle.dump(video_test, file)
file.close()